In [48]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline

def load_p(fname):
    return np.array(pickle.load(open(fname,"rb")))

In [49]:
X_train_cls = load_p("/scratch/pbanerj6/sml-dataset/X_train_cls.p")
X_val_cls = load_p("/scratch/pbanerj6/sml-dataset/X_val_cls.p")
X_test_cls = load_p("/scratch/pbanerj6/sml-dataset/X_test_cls.p")
X_train_all = load_p("/scratch/pbanerj6/sml-dataset/X_train_all_90.pkl")
X_val_all = load_p("/scratch/pbanerj6/sml-dataset/X_val_all_90.pkl")
X_test_all = load_p("/scratch/pbanerj6/sml-dataset/X_test_all_90.pkl")

X_train = np.concatenate([X_train_cls,X_train_all],axis=1)
X_val = np.concatenate([X_val_cls,X_val_all],axis=1)
X_test = np.concatenate([X_test_cls,X_test_all],axis=1)

In [50]:
y_train = load_p("/scratch/pbanerj6/sml-dataset/y_train.p")
y_val = load_p("/scratch/pbanerj6/sml-dataset/y_val.p")
y_test = load_p("/scratch/pbanerj6/sml-dataset/y_test.p")

In [51]:
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import os

In [52]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [53]:
train_dataset = MyDataset(X_train,y_train)
val_dataset = MyDataset(X_val,y_val)
test_dataset = MyDataset(X_test,y_test)

In [54]:
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=1,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=1,
    pin_memory=True
)

In [68]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
# N, D_in, H, D_out = 64, 768, 200, 2
N , D_in, H1, H2, D_out = 32, 6768, 500, 100, 2

In [69]:
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, D_out),
# )

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, D_out),
)

model.to('cuda')
model = torch.nn.DataParallel(model) 

loss_fn = nn.CrossEntropyLoss()


In [70]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [71]:
learning_rate = 1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

patience = 20

max_val_acc = 0.0
for t in tqdm(range(100),desc="Epoch"):
    model.train()
    total_loss = 0.0
    tmp_acc = 0.0
    steps = 0
    nb_eval_examples = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        steps+=1
        x=x.to('cuda')
        y=y.to('cuda')
        logits = model(x)
        loss = loss_fn(logits, y)
        logits = logits.detach().cpu().numpy()
        label_ids = y.to('cpu').numpy()
        tmp_acc += accuracy(logits, label_ids)
        # Compute and print loss.
        
        total_loss += loss.mean()
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
        nb_eval_examples += x.size(0)
    
    model.eval()
    val_acc = 0.0
    steps = 0
    nb_val_examples = 0
    for batch_idx, (x, y) in enumerate(val_loader):
        steps+=1
        logits = model(x)
        x=x.to('cuda')
        y=y.to('cuda')
        logits = logits.detach().cpu().numpy()
        label_ids = y.to('cpu').numpy()
        val_acc += accuracy(logits, label_ids)
        nb_val_examples += x.size(0)
    
    if max_val_acc < val_acc/nb_val_examples:
        patience = 20
        print("Saving Best Model!!")
        max_val_acc = val_acc/nb_val_examples
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join("/scratch/pbanerj6/sml-nn-v1", "best_model.bin")
        torch.save(model_to_save.state_dict(), output_model_file)
    else:
        patience-=1
        
    if patience == 0:
        print("Validation Accuracy Did Not Improve After", patience, "Epochs. Stopping.")
        break
    
    
    print("Epoch:",t,"Loss:",total_loss.item()/steps,"Train Accuracy:",(tmp_acc/nb_eval_examples),"Val Accuracy:",(val_acc/nb_val_examples))







Epoch:   0%|          | 0/100 [00:00<?, ?it/s]




Epoch:   1%|          | 1/100 [00:09<15:54,  9.64s/it]

Saving Best Model!!
Epoch: 0 Loss: 3.3673906042220745 Train Accuracy: 0.59124 Val Accuracy: 0.5886666666666667







Epoch:   2%|▏         | 2/100 [00:19<15:42,  9.62s/it]

Saving Best Model!!
Epoch: 1 Loss: 3.206304282330452 Train Accuracy: 0.6668533333333333 Val Accuracy: 0.6149333333333333







Epoch:   3%|▎         | 3/100 [00:28<15:33,  9.62s/it]

...

Epoch: 27 Loss: 1.2171439962184176 Train Accuracy: 0.9241333333333334 Val Accuracy: 0.6370666666666667







Epoch:  29%|██▉       | 29/100 [05:02<12:39, 10.70s/it]

Epoch: 28 Loss: 1.1698026616522605 Train Accuracy: 0.9285466666666666 Val Accuracy: 0.6310666666666667







Epoch:  30%|███       | 30/100 [05:13<12:28, 10.70s/it]

Epoch: 29 Loss: 1.1232398499833776 Train Accuracy: 0.9333466666666667 Val Accuracy: 0.6308
Validation Accuracy Did Not Improve After 0 Epochs. Stopping.


In [72]:
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, D_out),
# )

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, D_out),
)
output_model_file = os.path.join("/scratch/pbanerj6/sml-nn-v1", "best_model.bin")
model.load_state_dict(torch.load(output_model_file))
model.eval()

Sequential(
  (0): Linear(in_features=6768, out_features=500, bias=True)
  (1): ReLU()
  (2): Linear(in_features=500, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=2, bias=True)
)

In [73]:
from scipy.stats import rankdata
from scipy.special import softmax

def mrrs(out, labels):
#     print(out,labels)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        mrr+=1/rank
    return mrr/len(labels)

def precision_at(out,labels,prank=1):
    count = 0
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        if rank <= prank:
            count+=1
    return count/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels),precision_at(np.array(out),labels,1),precision_at(np.array(out),labels,3)

def load_ranking(fname):
    return pickle.load(open("/scratch/pbanerj6/sml-dataset/ranking_"+fname+".p","rb"))

qid2c,qid2indexmap = load_ranking("test")

model.to('cuda')
model = torch.nn.DataParallel(model) 
model.eval()
val_acc = 0.0
steps = 0
nb_val_examples = 0

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=1,
    pin_memory=1
)

all_preds = []

for batch_idx, (x, y) in enumerate(test_loader):
    x=x.to('cuda')
    y=y.to('cuda')
    logits = model(x)
    logits = logits.detach().cpu().numpy()
    
    for pred in logits:
        all_preds.append(softmax(pred))
    
    label_ids = y.to('cpu').numpy()
    val_acc += accuracy(logits, label_ids)
    nb_val_examples += x.size(0)
    
mrr = mrrwrapper(qid2c,qid2indexmap,all_preds)
print("Test Accuracy:",(val_acc/nb_val_examples),"MRR:",(mrr))

Test Accuracy: 0.6488 MRR: (0.6115571428571458, 0.3873333333333333, 0.8013333333333333)


In [40]:
# 100 : Test Accuracy: 0.6601333333333333 MRR: (0.6016777777777803, 0.38, 0.7846666666666666)
# 200 : Test Accuracy: 0.6572 MRR: (0.6074333333333362, 0.38666666666666666, 0.7813333333333333)
# 200-100 : Test Accuracy: 0.6604666666666666 MRR: (0.6049000000000025, 0.38333333333333336, 0.7853333333333333)
# 300-100 : Test Accuracy: 0.6583333333333333 MRR: (0.6061682539682567, 0.38666666666666666, 0.772)

# Both
# 300-100 , Test Accuracy: 0.6492666666666667 MRR: (0.6176015873015903, 0.398, 0.8046666666666666)
# 500-100, Test Accuracy: 0.6488 MRR: (0.6115571428571458, 0.3873333333333333, 0.8013333333333333) 

In [41]:
X_train.shape

(75000, 768)

In [2]:
import torch

In [5]:
a = torch.ones([100,100])
b = torch.ones([100,100])

In [11]:
torch.cat([a,b],1).size()

torch.Size([100, 200])